## ***Project 1: PINNs for parametric problems vs Greedy (Advection-Diffusion equation)*** ###

Train a PINN to solve the following parametric problem on
${\Omega} = (0, 1) \times (0, 1)$

$$
\begin{cases}
- \nabla \cdot ({\mu_1} \nabla u) + \boldsymbol{\beta}\cdot \nabla{u}= 0 & \text{in } \Omega,\\
{\mu_1} \nabla u \cdot \mathbf{n} = \mu_2 & \text{in } \Gamma_{b},\\
u = 0 & \text{in } \Gamma_{tl},\\
 \nabla u \cdot \mathbf{n} = 0 & \text{otherwise},
\end{cases}
$$

where $\mathbf{n}$ is the outer-normal to the portion of $\partial \Omega$ we consider.
Specifically, $\Gamma_b = [0, 1] \times \{0\}$ and $\Gamma_{tl} = ([0, 1] \times \{1\}) \cup (\{0\} \times [0,1])$.

The convection field is $\boldsymbol{\beta} = [y(1-y),0]^T$ and $(x,y)$ represent a spatial coordinate in $\Omega$.
The parametric space is $\mathcal P = [0.1, 10] \times [-1,1]$.


Compare the results with standard ROM based on Greedy approach, both in terms of online  accuracy with respect to the FE solutions, speedups and offline computations compared with the training phase.

# Dataset

## not balanced

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd

np.random.seed(2)

mu2 = np.random.uniform(low = -1., high = 1., size = (80,1))
mu1 = np.random.uniform(low = 0.1, high = 10., size = (47,1))

xv, yv = np.meshgrid(mu1, mu2)
xv = xv.reshape((xv.shape[0] * xv.shape[1] ))
yv = yv.reshape((yv.shape[0] * yv.shape[1] ))

df = pd.DataFrame(dict(mu1=xv,mu2=yv,ratio=abs(yv/xv)))

df['ratio_abs'] = df['ratio']

tol_ratio = 1.
df['ratio'] = df.mu2 / df.mu1
df['great'] = np.array(df.ratio_abs >= tol_ratio)

df.describe(include='all')

,mu1,mu2,ratio,ratio_abs,great
count,3760.000000,3760.000000,3760.000000,3760.000000,3760
unique,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,3626
mean,4.897154,-0.082362,-0.037603,0.202691,NaN
std,2.621930,0.533643,0.503487,0.462406,NaN
min,0.201475,-0.973965,-4.834165,0.000002,NaN
25%,2.533531,-0.486794,-0.106544,0.034020,NaN
50%,5.447896,-0.127258,-0.021577,0.088984,NaN
75%,6.655800,0.239473,0.061779,0.167439,NaN


In [ ]:
# x and y given as array_like objects
import plotly.express as px
import plotly.graph_objects as go

fig = px.box(df.mu1,points="all")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()
fig = px.box(df.mu2,points="all")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)
fig.show()

import matplotlib.pyplot as plt



fig = px.scatter(df, x="mu1", y="mu2", color = "ratio")
fig.update_layout(
    xaxis_title="mu1",
    yaxis_title="mu2",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.show()

fig = px.box(df.ratio_abs,points="all")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

## More representative

In [2]:
!git clone https://github.com/gioanateora/Project_MOR.git

fatal: destination path 'Project_MOR' already exists and is not an empty directory.
/content/Project_MOR


In [4]:
from google.colab import files
import numpy as np
import pandas as pd

np.random.seed(2)

#mu2 = np.random.uniform(low = -1., high = 1., size = (80,1))
#mu1 = np.random.exponential(scale=1.0, size=(50,1))
#mu1 = mu1[(mu1 >= 0.1)]
#mu1 = mu1[(mu1 <= 10)]

#xv, yv = np.meshgrid(mu1, mu2)
#xv = xv.reshape((xv.shape[0] * xv.shape[1] ))
#yv = yv.reshape((yv.shape[0] * yv.shape[1] ))

#df = pd.DataFrame(dict(mu1=xv,mu2=yv,ratio=abs(yv/xv)))
#df.describe()

#df.to_csv("data.csv",sep = ';', header=True)
#files.download('./data.csv')

df = pd.read_csv('/content/Project_MOR/data.csv', sep=';')
df['ratio_abs'] = df['ratio']

tol_ratio = 1.
df['ratio'] = df.mu2 / df.mu1
df['great'] = np.array(df.ratio_abs >= tol_ratio)

df.describe(include='all')

,Unnamed: 0,mu1,mu2,ratio,ratio_abs,great
count,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760
unique,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,2661
mean,1879.500000,0.896672,-0.082362,-0.182781,0.985233,NaN
std,1085.562834,0.726157,0.533643,1.612634,1.289598,NaN
min,0.000000,0.117189,-0.973965,-8.311046,0.000005,NaN
25%,939.750000,0.320287,-0.486794,-0.680883,0.193948,NaN
50%,1879.500000,0.783828,-0.127258,-0.119690,0.530308,NaN
75%,2819.250000,1.085307,0.239473,0.334116,1.159774,NaN


In [5]:
# x and y given as array_like objects
import plotly.express as px
import plotly.graph_objects as go

fig = px.box(df.mu1,points="all")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()
fig = px.box(df.mu2,points="all")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)
fig.show()

import matplotlib.pyplot as plt



fig = px.scatter(df, x="mu1", y="mu2", color = "ratio")
fig.update_layout(
    xaxis_title="mu1",
    yaxis_title="mu2",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.show()

fig = px.box(df.ratio_abs,points="all")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

### Split test and train

In [6]:
train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index)
train.describe(include='all')

,Unnamed: 0,mu1,mu2,ratio,ratio_abs,great
count,3008.000000,3008.000000,3008.000000,3008.000000,3008.000000,3008
unique,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,2134
mean,1878.225066,0.902807,-0.085273,-0.200700,0.986252,NaN
std,1094.726990,0.736947,0.531337,1.611568,1.290127,NaN
min,0.000000,0.117189,-0.973965,-8.311046,0.000005,NaN
25%,927.750000,0.320287,-0.505646,-0.682189,0.196121,NaN
50%,1882.000000,0.783828,-0.128010,-0.123232,0.525531,NaN
75%,2824.250000,1.085307,0.238542,0.316447,1.149424,NaN


In [7]:
test.describe(include='all')

,Unnamed: 0,mu1,mu2,ratio,ratio_abs,great
count,752.000000,752.000000,752.000000,752.000000,752.000000,752
unique,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,527
mean,1884.599734,0.872130,-0.070717,-0.111107,0.981159,NaN
std,1048.799983,0.681209,0.542968,1.615976,1.288326,NaN
min,17.000000,0.117189,-0.973965,-8.090738,0.000007,NaN
25%,983.750000,0.349802,-0.480510,-0.653605,0.189687,NaN
50%,1862.500000,0.776945,-0.126505,-0.088095,0.552062,NaN
75%,2790.250000,1.083176,0.251904,0.370512,1.196361,NaN


# PINN

In [ ]:
#### starting stuff ####

import torch
import torch.nn as nn
from torch.autograd import Variable
import math

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_nodes = 30
        self.input_layer = nn.Linear(4, num_nodes)
        torch.nn.init.xavier_normal_(self.input_layer.weight)
        self.hidden_layer1 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer1.weight)
        self.hidden_layer2 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer2.weight)
        self.hidden_layer3 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer3.weight)
        self.output_layer = nn.Linear(num_nodes, 1)
        torch.nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, x, y, mu1, mu2):
        input = torch.cat([x, y, mu1, mu2],axis=1) # combines the column array
        layer1_out = torch.tanh(self.input_layer(input))
        layer2_out = torch.tanh(self.hidden_layer1(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer2(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer3(layer3_out))
        output = self.output_layer(layer4_out)
        return x * (1-y) * output


## PDE as loss function. Thus would use the network which we call as u_theta
def R(x, y, mu1, mu2, net):
    u = net(x, y, mu1, mu2)
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]

    u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]

    pde = - mu1 * (u_xx + u_yy) + y * (1-y) * u_x

    return pde

def Neumann_bottom(pt_x_bc_N_bottom, pt_y_bc_N_bottom, pt_mu1_N_bottom, pt_mu2_N_bottom, net):
    u = net(pt_x_bc_N_bottom, pt_y_bc_N_bottom, pt_mu1_N_bottom, pt_mu2_N_bottom)
    u_x = torch.autograd.grad(u.sum(), pt_x_bc_N_bottom, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), pt_y_bc_N_bottom, create_graph=True)[0]
    neumann = - pt_mu1_N_bottom * u_y - pt_mu2_N_bottom
    return neumann

def Neumann_right(pt_x_bc_N_right, pt_y_bc_N_right, pt_mu1_N_right, pt_mu2_N_right, net):
    u = net(pt_x_bc_N_right,pt_y_bc_N_right, pt_mu1_N_right, pt_mu2_N_right)
    u_x = torch.autograd.grad(u.sum(), pt_x_bc_N_right, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), pt_y_bc_N_right, create_graph=True)[0]
    neumann = u_x
    return neumann

def compute_l2_loss(w):
    return torch.square(w).sum()

from numpy.core.multiarray import ndarray
import matplotlib.pyplot as plt
from scipy.stats import qmc

np.random.seed(2)
random_seed = 1
torch.manual_seed(random_seed)

n_N_bottom = 2000
n_N_right = 2000
num_points = 4000

tol = 1.0e-07,
mtol = 1. - 1.0e-07,


# Neuman: right
x_bc_N_right = np.ones((n_N_right,1))
y_bc_N_right = np.ravel(np.linspace(tol, mtol, num = n_N_right)).reshape(-1,1)


# Neumann: bottom
x_bc_N_bottom = np.ravel(np.linspace(tol, mtol, num = n_N_bottom)).reshape(-1,1)
y_bc_N_bottom = np.zeros((n_N_bottom ,1))

### (2) Model
net = Net()
mse_cost_function = torch.nn.MSELoss() # Mean squared error
#optimizer = torch.optim.Adam(net.parameters())

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
#lambda1 = lambda epoch: 0.7 ** math.floor(epoch / 5000)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

### (3) Training / Fitting

print("######### TRAIN: PINN #############")
loss_vector = []
lrs = []

iterations = 30000
for epoch in range(iterations):
    optimizer.zero_grad() # to make the gradients zero

    # Loss based on boundary conditions: Neumann right
    minibatch=train.sample(n = n_N_right, replace=True)
    mu1_N_right = np.ravel(np.array(minibatch.mu1)).reshape(-1,1)
    mu2_N_right = np.ravel(np.array(minibatch.mu2)).reshape(-1,1)
    pt_mu1_N_right = Variable(torch.from_numpy(mu1_N_right).float(), requires_grad=False)
    pt_mu2_N_right = Variable(torch.from_numpy(mu2_N_right).float(), requires_grad=False)

    pt_x_bc_N_right = Variable(torch.from_numpy(x_bc_N_right).float(), requires_grad=True)
    pt_y_bc_N_right = Variable(torch.from_numpy(y_bc_N_right).float(), requires_grad=True)

    net_neumann_right = Neumann_right(pt_x_bc_N_right, pt_y_bc_N_right, pt_mu1_N_right, pt_mu2_N_right, net)
    zero_n_right = np.zeros((n_N_right , 1))
    pt_zero_n_right = Variable(torch.from_numpy(zero_n_right).float(), requires_grad=False)
    mse_bc_N_right = mse_cost_function(net_neumann_right, pt_zero_n_right)

    # Loss based on boundary conditions: Neumann bottom
    minibatch=train.sample(n = n_N_bottom, replace=True)
    mu1_N_bottom = np.ravel(np.array(minibatch.mu1)).reshape(-1,1)
    mu2_N_bottom = np.ravel(np.array(minibatch.mu2)).reshape(-1,1)
    pt_mu1_N_bottom = Variable(torch.from_numpy(mu1_N_bottom).float(), requires_grad=False)
    pt_mu2_N_bottom = Variable(torch.from_numpy(mu2_N_bottom).float(), requires_grad=False)

    pt_x_bc_N_bottom = Variable(torch.from_numpy(x_bc_N_bottom).float(), requires_grad=True)
    pt_y_bc_N_bottom = Variable(torch.from_numpy(y_bc_N_bottom).float(), requires_grad=True)

    net_neumann_bottom = Neumann_bottom(pt_x_bc_N_bottom, pt_y_bc_N_bottom, pt_mu1_N_bottom, pt_mu2_N_bottom, net)
    zero_n_bottom = np.zeros((n_N_bottom ,1))
    pt_zero_n_bottom = Variable(torch.from_numpy(zero_n_bottom).float(), requires_grad=False)
    mse_bc_N_bottom = mse_cost_function(net_neumann_bottom, pt_zero_n_bottom)

    # Loss based on PDE
    minibatch=train.sample(n = num_points, replace=True)
    mu1_collocation = np.ravel(np.array(minibatch.mu1)).reshape(-1,1)
    mu2_collocation = np.ravel(np.array(minibatch.mu2)).reshape(-1,1)
    pt_mu1_collocation = Variable(torch.from_numpy(mu1_collocation).float(), requires_grad=False)
    pt_mu2_collocation = Variable(torch.from_numpy(mu2_collocation).float(), requires_grad=False)

    x_collocation = np.random.uniform(low=tol, high=mtol, size=(num_points,1))
    y_collocation = np.random.uniform(low=tol, high=mtol, size=(num_points,1))
    all_zeros = np.zeros((num_points,1))

    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True)
    pt_y_collocation = Variable(torch.from_numpy(y_collocation).float(), requires_grad=True)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False)

    f_out = R(pt_x_collocation, pt_y_collocation, pt_mu1_collocation, pt_mu2_collocation, net) # output of R(x,t)
    mse_f = mse_cost_function(f_out, pt_all_zeros)

    # Combining the loss functions
    loss = mse_bc_N_bottom + mse_bc_N_right + mse_f

    #l2_lambda = 0.0001
    #l2_regularization = torch.tensor(0., requires_grad=True)
    #for name, param in net.named_parameters():
    #    if 'bias' not in name:
    #      with torch.no_grad():
    #        l2_regularization += torch.sum(param*param)
    #loss += l2_lambda * l2_regularization

    loss.backward()
    optimizer.step()
    loss_vector.append(loss.item())
    #lrs.append(optimizer.param_groups[0]["lr"])


    with torch.autograd.no_grad():
      if( epoch % 200 == 0):
        print(epoch,"Loss:",loss.item(), "lr:", optimizer.param_groups[0]["lr"] )

    #scheduler.step()

torch.save(net.state_dict(), 'model_30000_NN_2000_Col_4000_wr.pt')

df = pd.DataFrame(dict(loss=loss_vector))
df.describe()

df.to_csv("data_nn_30000_NN_2000_Col_4000_wr.csv",sep = ';', header=True)

In [ ]:
df = pd.read_csv('/content/Project_MOR/data_nn_30000_NN_2000_Col_4000_wr.csv', sep=';')
loss_vector = df['loss']
fig = px.scatter()
fig.add_scatter(x=np.arange(0, len(loss_vector)), y=loss_vector, name= "loss")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_yaxes(type="log")

fig.show()

In [ ]:
def draw_neural_net(ax, left, right, bottom, top, layer_sizes, edgewidth):
    '''
    Draw a neural network cartoon using matplotilb.

    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here
        - layer_sizes : list of int
            List of layer sizes, including input and output dimensionality
        - edgewidth : linewidth parameters used to draw the edges
    '''
    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        add_bias = 1 if n < n_layers-1 else 0
        for m in range(layer_size+add_bias):
            if m == 0 and add_bias:
              rectangle = plt.Rectangle((n*h_spacing + left - v_spacing/4., layer_top - (m-add_bias)*v_spacing - v_spacing/4.),
                                        2*v_spacing/4., 2*v_spacing/4., fc='red',ec="red")
              ax.add_artist(rectangle)
            else:
              circle = plt.Circle((n*h_spacing + left, layer_top - (m-add_bias)*v_spacing), v_spacing/4.,
                                  color='b', ec='k', zorder=4)
              ax.add_artist(circle)
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        add_bias = 1 if n < n_layers-1 else 0
        # for m in range(layer_size_a):
        for m in range(layer_size_a+1):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - (m-add_bias)*v_spacing, layer_top_b - o*v_spacing], c='k', linewidth=edgewidth)
                ax.add_artist(line)



fig = plt.figure(figsize=(60, 60))
ax = fig.gca()
ax.axis('off')
draw_neural_net(ax, .1, .9, .1, .9, layer_sizes=[4, 30, 30, 30, 30, 1], edgewidth=1)

fig.savefig('nn.png')

Output hidden; open in https://colab.research.google.com to view.

# ROM - GREEDY

In [8]:
!git clone https://github.com/fvicini/CppToPython.git
%cd CppToPython

!git submodule init
!git submodule update

!mkdir -p externals
%cd externals
!cmake -DINSTALL_VTK=OFF -DINSTALL_LAPACK=OFF ../gedim/3rd_party_libraries
!make -j4
%cd ..

!mkdir -p release
%cd release
!cmake -DCMAKE_PREFIX_PATH="/content/CppToPython/externals/Main_Install/eigen3;/content/CppToPython/externals/Main_Install/triangle;/content/CppToPython/externals/Main_Install/tetgen;/content/CppToPython/externals/Main_Install/googletest" ../
!make -j4 GeDiM4Py
%cd ..

Cloning into 'CppToPython'...
remote: Enumerating objects: 645, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 645 (delta 31), reused 50 (delta 17), pack-reused 487
Receiving objects: 100% (645/645), 596.41 KiB | 9.32 MiB/s, done.
Resolving deltas: 100% (424/424), done.
/content/CppToPython
Submodule 'gedim' (https://github.com/fvicini/gedim.git) registered for path 'gedim'
Cloning into '/content/CppToPython/gedim'...
Submodule path 'gedim': checked out 'c52847f1a527d36b529f39d3744afe9fd612b62c'
/content/CppToPython/externals
-- The CXX compiler identification is GNU 9.4.0
-- The C compiler identification is GNU 9.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for workin

In [9]:
import numpy as np
import GeDiM4Py as gedim
lib = gedim.ImportLibrary("./release/GeDiM4Py.so")

geometricTolerance = 1.0e-8
config = { 'GeometricTolerance': geometricTolerance }
gedim.Initialize(config, lib)

meshSize = 0.0001
order = 1

domain = { 'SquareEdge': 1.0, 'VerticesBoundaryCondition': [1,0,1,1], 'EdgesBoundaryCondition': [2,3,1,1], 'DiscretizationType': 1, 'MeshCellsMaximumArea': meshSize }
[meshInfo, mesh] = gedim.CreateDomainSquare(domain, lib)

#gedim.PlotMesh(mesh)

df_mesh = pd.DataFrame(mesh.T)
#df_mesh.to_csv('mesh0001.csv', sep = ';', header=True)
#files.download('./mesh0001.csv')

In [10]:
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 2, 3, 3] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)


def Poisson_beta(numPoints, points):
    matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
    values = np.zeros((2, numPoints))
    values[0,:] = (1.0 - matPoints[1,:]) * matPoints[1,:]
    return values.ctypes.data

def Poisson_f(numPoints, points):
    values = np.zeros(numPoints)
    return values.ctypes.data

def Poisson_strongTerm(numPoints, points):
    values = np.zeros(numPoints)
    return values.ctypes.data

def Poisson_diffusion(numPoints, points):
    values = np.ones(numPoints)
    return values.ctypes.data

def Poisson_weakTerm_bottom(numPoints, points):
    values = np.ones(numPoints)
    return values.ctypes.data

[stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Poisson_diffusion, problemData, lib)
[advection, advectionStrong] = gedim.AssembleAdvectionMatrix(Poisson_beta, problemData, lib)
forcingTerm = gedim.AssembleForcingTerm(Poisson_f, problemData, lib)
solutionStrong = gedim.AssembleStrongSolution(Poisson_strongTerm, 1, problemData, lib)
weakTerm_bottom = gedim.AssembleWeakTerm(Poisson_weakTerm_bottom, 2, problemData, lib)

In [11]:
#### inner product X
# ||grad(u)||^2
X = stiffness

### define the problem
AQH = [stiffness, advection]
fQH = [weakTerm_bottom]

def thetaA(mu):
    return [mu[0], 1.0]
def thetaF(mu):
    return [mu[1]]

In [12]:
### define the training set
np.random.seed(1)

M = 100

minibatch=train.sample(n = M)
minibatch.describe()

,Unnamed: 0,mu1,mu2,ratio,ratio_abs
count,100.00000,100.000000,100.000000,100.000000,100.000000
mean,1905.50000,0.842735,-0.064120,-0.133740,1.240637
std,1077.76536,0.740761,0.555795,2.044038,1.625248
min,5.00000,0.117189,-0.973965,-8.311046,0.000014
25%,1050.25000,0.241608,-0.486794,-0.769700,0.224187
50%,1972.50000,0.608089,-0.126505,-0.151490,0.554457
75%,2748.00000,1.071520,0.310986,0.458643,1.627124
max,3756.00000,3.608616,0.987704,8.245695,8.311046


In [ ]:
from scipy.sparse.linalg import splu

### define the training set

np.random.seed(1)

training_set = np.concatenate((np.ravel(np.array(minibatch['mu1'])).reshape(-1,1), np.ravel(np.array(minibatch['mu2'])).reshape(-1,1)), axis=1)

def normX(v, X):
	return np.sqrt(np.transpose(v) @ X @ v)

def ProjectSystem(AQH, fQH, B):
    AQN = []
    fQN = []
    for AH in AQH:
        AQN.append(np.copy(np.transpose(B) @ AH @ B))
    for fH in fQH:
        fQN.append(np.copy(np.transpose(B) @ fH))
    return [AQN, fQN]

def Solve_full_order(AQH, fQH, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQH[0]
    f = thetaF_mu[0] * fQH[0]
    for i in range(1, len(AQH)):
        A += thetaA_mu[i] * AQH[i]
    for i in range(1, len(fQH)):
        f += thetaF_mu[i] * fQH[i]
    return gedim.LUSolver(A, f, lib)

def Solve_reduced_order(AQN, fQN, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQN[0]
    f = thetaF_mu[0] * fQN[0]
    for i in range(1, len(AQN)):
        A += thetaA_mu[i] * AQN[i]
    for i in range(1, len(fQN)):
        f += thetaF_mu[i] * fQN[i]
    return np.linalg.solve(A, f)

def OfflineResidual(AQH, fQH, B, invX):
    Cq1q2 = []
    dq1q2 = []
    Eq1q2 = []

    for q1 in range(0, len(AQH)):
        Z = invX.solve(AQH[q1] @ B)

        aqh_list = []
        for q2 in range(0, len(AQH)):
            aqh_list.append(np.copy(np.transpose(Z) @ AQH[q2] @ B))
        Eq1q2.append(aqh_list.copy())

        fqh_list = []
        for q2 in range(0, len(fQH)):
            fqh_list.append(np.copy(np.transpose(Z) @ fQH[q2]))
        dq1q2.append(fqh_list.copy())

    for q1 in range(0, len(fQH)):
        t = invX.solve(fQH[q1])

        fqh_list = []
        for q2 in range(0, len(fQH)):
            fqh_list.append(np.copy(np.transpose(t) @ fQH[q2]))
        Cq1q2.append(fqh_list.copy())

    return [Cq1q2, dq1q2, Eq1q2]

def InfSupConstant(mu):
    return thetaA(mu)[0]

def ErrorEstimate(Cq1q2, dq1q2, Eq1q2, thetaA_mu, thetaF_mu, solN, beta):
    fError = 0.0
    for q1 in range(0, len(Cq1q2)):
        for q2 in range(0, len(Cq1q2[q1])):
            fError += thetaF_mu[q1] * thetaF_mu[q2] * Cq1q2[q1][q2]

    uError = 0.0
    for q1 in range(0, len(Eq1q2)):
        for q2 in range(0, len(Eq1q2[q1])):
            uError += thetaA_mu[q1] * thetaA_mu[q2] * np.transpose(solN) @ Eq1q2[q1][q2] @ solN

    fuError = 0.0
    for q1 in range(0, len(dq1q2)):
        for q2 in range(0, len(dq1q2[q1])):
            fuError += thetaA_mu[q1] * thetaF_mu[q2] * np.transpose(solN) @ dq1q2[q1][q2]

    deltaN_squared = fError - 2.0 * fuError + uError
    if abs(deltaN_squared) <= 1.0e-12: # protect cancellation error
        deltaN_squared = 0.0
    if deltaN_squared < -1.0e-12:
      print(deltaN_squared)
      raise Exception('deltaN_squared is negative')

    return np.sqrt(deltaN_squared) / beta

def GramSchmidt(V, u, X):
    z = u
    if np.size(V) > 0:
        z = u - V @ (np.transpose(V) @ (X @ u))
    return z / normX(z, X)

chosen_mu = []

##### Greedy #####
def Greedy(AQH, fQH, X, N_max, tol):
    N = 0
    basis_functions = []
    B = np.empty((0,0))
    deltaN = tol + 1.
    training_set_list = training_set.tolist()
    initial_muN = np.random.choice(len(training_set_list) - 1, 1)[0]
    muN = training_set_list.pop(initial_muN)
    invX = splu(X)

    print('Perfom greedy algorithm...')
    while len(training_set_list) > 0 and N < N_max and deltaN > tol:
        N = N + 1
        chosen_mu.append(muN)
        print('\t', N,'/', N_max, '-', '{:.16e}'.format(np.mean(deltaN)), '/', '{:.16e}'.format(np.mean(tol)))
        snapshot = Solve_full_order(AQH, fQH, thetaA(muN), thetaF(muN))
        basis_function = GramSchmidt(B, snapshot, X)
        basis_functions.append(np.copy(basis_function))
        B = np.transpose(np.array(basis_functions))
        BX = np.transpose(B) @ X @ B

        [AQN, fQN] = ProjectSystem(AQH, fQH, B)
        [Cq1q2, dq1q2, Eq1q2] = OfflineResidual(AQH, fQH, B, invX)

        counter = 0
        mu_selected_index = -1
        max_deltaN = -1.
        for mu in training_set_list:
            solN_mu = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
            betaN_mu = InfSupConstant(mu)
            deltaN_mu = ErrorEstimate(Cq1q2, dq1q2, Eq1q2, thetaA(mu), thetaF(mu), solN_mu, betaN_mu)

            if deltaN_mu > max_deltaN:
                max_deltaN = deltaN_mu
                mu_selected_index = counter

            counter = counter + 1

        if mu_selected_index == -1:
            raise Exception('ERROR, parameter not found')

        muN = training_set_list.pop(mu_selected_index)
        deltaN = max_deltaN

    return [N, np.transpose(np.array(basis_functions))]

### Compute Greedy
tol = 1.0e-5
N_max = 40

[N_Greedy, B_Greedy] = Greedy(AQH, fQH, X, N_max, tol)
print("N_Greedy", N_Greedy)

[AQN_Greedy, fQN_Greedy] = ProjectSystem(AQH, fQH, B_Greedy)

Perfom greedy algorithm...
	 1 / 40 - 1.0000100000000001e+00 / 1.0000000000000001e-05
	 2 / 40 - 3.9337162234875073e-01 / 1.0000000000000001e-05
	 3 / 40 - 9.5036206828010353e-03 / 1.0000000000000001e-05
	 4 / 40 - 1.1231791721583856e-04 / 1.0000000000000001e-05
N_Greedy 4


In [ ]:
print(np.array(chosen_mu)[:,1]/ np.array(chosen_mu)[:,0])

[-0.14390543 -8.31104644  4.53841099 -6.31802183]


In [ ]:
print(B_Greedy.shape)

(7757, 4)


# Comparison

In [ ]:
!pip install pyevtk

import numpy as np
from pyevtk.hl import pointsToVTK, unstructuredGridToVTK
from pyevtk.vtk import VtkPolygon, VtkLine

def export_cells_0(path_file: str, x: float, y: float, point_data):
    pointsToVTK(path_file, x, y, np.zeros(x.shape), data=point_data)

def export_solution(path_file: str, PINN_solution, ROM_solution, FOM_solution, x, y):
    point_data = {"PINN_solution": PINN_solution, "ROM_solution": ROM_solution, "FEM_solution": FOM_solution }
    export_cells_0(path_file, x, y, point_data)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_nodes = 30
        self.input_layer = nn.Linear(4, num_nodes)
        torch.nn.init.xavier_normal_(self.input_layer.weight)
        self.hidden_layer1 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer1.weight)
        self.hidden_layer2 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer2.weight)
        self.hidden_layer3 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer3.weight)
        self.output_layer = nn.Linear(num_nodes, 1)
        torch.nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, x, y, mu1, mu2):
        input = torch.cat([x, y, mu1, mu2],axis=1) # combines the column array
        layer1_out = torch.tanh(self.input_layer(input))
        layer2_out = torch.tanh(self.hidden_layer1(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer2(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer3(layer3_out))
        output = self.output_layer(layer4_out)
        return x * (1-y) * output

net = Net()
net.load_state_dict(torch.load('/content/Project_MOR/model_30000_NN_2000_Col_4000_wr.pt',))
net.eval()

Net(
  (input_layer): Linear(in_features=4, out_features=30, bias=True)
  (hidden_layer1): Linear(in_features=30, out_features=30, bias=True)
  (hidden_layer2): Linear(in_features=30, out_features=30, bias=True)
  (hidden_layer3): Linear(in_features=30, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=1, bias=True)
)

In [ ]:
from numpy import linalg
import math
import time

num_mu1 = len(test.mu1)

errors_PINN = np.zeros(num_mu1)
errors_inf_PINN = np.zeros(num_mu1)
speed_up_PINN = np.zeros(num_mu1)

errors_ROM = np.zeros(num_mu1)
errors_inf_ROM = np.zeros(num_mu1)
speed_up_ROM = np.zeros(num_mu1)

max_abs_NN_sol = np.zeros(num_mu1)
max_NN_sol = np.zeros(num_mu1)
min_NN_sol = np.zeros(num_mu1)

max_abs_FOM_sol = np.zeros(num_mu1)
max_FOM_sol = np.zeros(num_mu1)
min_FOM_sol = np.zeros(num_mu1)

max_abs_ROM_sol = np.zeros(num_mu1)
max_ROM_sol = np.zeros(num_mu1)
min_ROM_sol = np.zeros(num_mu1)
df_mesh = pd.read_csv('/content/Project_MOR/mesh0001.csv', sep=';')

count = 0
for (mu1,mu2) in zip(test.mu1, test.mu2):

  mu = [mu1, mu2]
  x = np.array(df_mesh[df_mesh.columns[1]])
  y = np.array(df_mesh[df_mesh.columns[2]])

  # PINN solution
  x_reshape = np.ravel(x).reshape(-1,1)
  y_reshape = np.ravel(y).reshape(-1,1)

  start_PINN = time.time()
  pt_mu1_test = Variable(torch.from_numpy(mu1 * np.ones((x.shape[0], 1))).float(), requires_grad=True)
  pt_mu2_test = Variable(torch.from_numpy(mu2 * np.ones((x.shape[0], 1))).float(), requires_grad=True)
  pt_x = Variable(torch.from_numpy(x_reshape).float(), requires_grad=True)
  pt_y = Variable(torch.from_numpy(y_reshape).float(), requires_grad=True)
  pt_u = net(pt_x,pt_y, pt_mu1_test, pt_mu2_test)
  time_PINN = time.time() - start_PINN

  nn_solution = np.squeeze(pt_u.detach().numpy())
  max_abs_NN_sol[count] = np.absolute(nn_solution).max()
  max_NN_sol[count] = nn_solution.max()
  min_NN_sol[count] = nn_solution.min()

  ##### full #####
  start_fom = time.time()
  full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))

  fem_solution = np.zeros(x.shape)
  count_dof = 0
  count_strong = 0
  for i in range(len(fem_solution)):
      if abs(x[i]) < geometricTolerance or abs(y[i] - 1.) < geometricTolerance:
          fem_solution[i] = 0.
          count_strong += 1
      else:
          fem_solution[i] = full_solution[count_dof]
          count_dof += 1

  #print(count_dof)

  time_fom = time.time() - start_fom

  max_abs_FOM_sol[count] = np.absolute(fem_solution).max()
  max_FOM_sol[count] = fem_solution.max()
  min_FOM_sol[count] = fem_solution.min()


  #### reduced #####
  start_rom = time.time()
  reduced_solution = Solve_reduced_order(AQN_Greedy, fQN_Greedy, thetaA(mu), thetaF(mu))

  proj_reduced_solution = B_Greedy @ reduced_solution

  rom_solution = np.zeros(x.shape)
  count_dof = 0
  count_strong = 0
  for i in range(len(rom_solution)):
      if abs(x[i]) < geometricTolerance or abs(y[i] - 1.) < geometricTolerance:
          rom_solution[i] = 0.
          count_strong += 1
      else:
          rom_solution[i] = proj_reduced_solution[count_dof]
          count_dof += 1

  time_rom = time.time() - start_rom

  max_abs_ROM_sol[count] = np.absolute(rom_solution).max()
  max_ROM_sol[count] = rom_solution.max()
  min_ROM_sol[count] = rom_solution.min()

  # compute errors
  speed_up_PINN[count] = time_fom / time_PINN
  errors_PINN[count] = linalg.norm(nn_solution - fem_solution)**2 / len((fem_solution))
  errors_inf_PINN[count] = linalg.norm((nn_solution - fem_solution), ord = np.inf)

  speed_up_ROM[count] = time_fom / time_rom
  errors_ROM[count] = linalg.norm(proj_reduced_solution - full_solution)**2 / len((full_solution))
  errors_inf_ROM[count] = linalg.norm((proj_reduced_solution - full_solution), ord = np.inf)

  count +=1

In [ ]:
d = {'MSE_PINN': errors_PINN, 'Err_Inf_PINN': errors_inf_PINN, 'Speed_Up_PINN': speed_up_PINN,'MSE_ROM': errors_ROM, 'Err_Inf_ROM': errors_inf_ROM, 'Speed_Up_ROM': speed_up_ROM}
df = pd.DataFrame(data=d)

df.describe().apply(lambda s: s.apply('{0:.2e}'.format))

,MSE_PINN,Err_Inf_PINN,Speed_Up_PINN,MSE_ROM,Err_Inf_ROM,Speed_Up_ROM
count,7.52e+02,7.52e+02,7.52e+02,7.52e+02,7.52e+02,7.52e+02
mean,4.33e-03,4.59e-02,1.26e+01,8.69e-15,1.27e-07,5.72e+00
std,2.11e-02,1.02e-01,3.52e+00,1.73e-14,1.51e-07,1.61e+00
min,6.90e-08,7.78e-04,1.77e+00,3.13e-39,1.19e-19,2.19e+00
25%,1.30e-06,6.34e-03,1.13e+01,2.94e-17,1.13e-08,5.31e+00
50%,7.24e-06,1.49e-02,1.29e+01,1.09e-15,6.98e-08,5.85e+00
75%,3.31e-04,3.09e-02,1.37e+01,8.11e-15,1.92e-07,6.06e+00
max,2.18e-01,7.88e-01,4.69e+01,9.65e-14,6.59e-07,2.05e+01


In [ ]:
test['max_FOM'] = max_FOM_sol
test['min_FOM'] = min_FOM_sol
test['max_FOM_abs'] = max_abs_FOM_sol

fig = px.scatter()
fig.add_scatter(x=test.ratio, y=test.min_FOM, mode='markers', name= "min")
fig.add_scatter(x=test.ratio, y=test.max_FOM, mode='markers', name= "max")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_layout(
    xaxis_title="mu2/mu1",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [ ]:
test['max_PINN'] = max_NN_sol
test['min_PINN'] = min_NN_sol
test['max_PINN_abs'] = max_abs_NN_sol

fig = px.scatter()
fig.add_scatter(x=test.ratio, y=test.min_PINN, mode='markers', name= "min")
fig.add_scatter(x=test.ratio, y=test.max_PINN, mode='markers', name= "max")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    xaxis_title="mu2/mu1",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [ ]:
test['max_ROM'] = max_ROM_sol
test['min_ROM'] = min_ROM_sol
test['max_ROM_abs'] = max_abs_ROM_sol

fig = px.scatter()
fig.add_scatter(x=test.ratio, y=test.min_ROM, mode='markers', name= "min")
fig.add_scatter(x=test.ratio, y=test.max_ROM, mode='markers', name= "max")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    xaxis_title="mu2/mu1",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [ ]:
from numpy import linalg
import math
import time

mu1 = 1
mu2 = 0.5
mu = [mu1, mu2]
x = np.array(df_mesh[df_mesh.columns[1]])
y = np.array(df_mesh[df_mesh.columns[2]])

# PINN solution
x_reshape = np.ravel(x).reshape(-1,1)
y_reshape = np.ravel(y).reshape(-1,1)


pt_mu1_test = Variable(torch.from_numpy(mu1 * np.ones((x.shape[0], 1))).float(), requires_grad=True)
pt_mu2_test = Variable(torch.from_numpy(mu2 * np.ones((x.shape[0], 1))).float(), requires_grad=True)
pt_x = Variable(torch.from_numpy(x_reshape).float(), requires_grad=True)
pt_y = Variable(torch.from_numpy(y_reshape).float(), requires_grad=True)
pt_u = net(pt_x,pt_y, pt_mu1_test, pt_mu2_test)
nn_solution = np.squeeze(pt_u.detach().numpy())


##### full #####
full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))

fem_solution = np.zeros(x.shape)
count_dof = 0
count_strong = 0
for i in range(len(fem_solution)):
    if abs(x[i]) < geometricTolerance or abs(y[i] - 1.) < geometricTolerance:
        fem_solution[i] = 0.
        count_strong += 1
    else:
        fem_solution[i] = full_solution[count_dof]
        count_dof += 1


#### reduced #####
reduced_solution = Solve_reduced_order(AQN_Greedy, fQN_Greedy, thetaA(mu), thetaF(mu))

proj_reduced_solution = B_Greedy @ reduced_solution

rom_solution = np.zeros(x.shape)
count_dof = 0
count_strong = 0
for i in range(len(rom_solution)):
    if abs(x[i]) < geometricTolerance or abs(y[i] - 1.) < geometricTolerance:
        rom_solution[i] = 0.
        count_strong += 1
    else:
        rom_solution[i] = proj_reduced_solution[count_dof]
        count_dof += 1

export_solution('./solutionFig', nn_solution, rom_solution, fem_solution, x, y)
from google.colab import files
files.download('./solutionFig.vtu')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# A toy experiment

In [14]:
train_wt = train[train['ratio_abs'] <= 1]
train_wt.describe()

,Unnamed: 0,mu1,mu2,ratio,ratio_abs
count,2134.000000,2134.000000,2134.000000,2134.000000,2134.000000
mean,1828.329897,1.132282,-0.067826,-0.071432,0.368312
std,1095.616072,0.752793,0.446390,0.456609,0.279070
min,0.000000,0.117189,-0.973965,-0.999881,0.000005
25%,866.250000,0.671907,-0.339330,-0.385334,0.126459
50%,1796.500000,0.977005,-0.064425,-0.071032,0.307221
75%,2794.500000,1.381418,0.160008,0.207615,0.582361
max,3757.000000,3.608616,0.987704,0.995395,0.999881


In [ ]:
#### starting stuff ####

import torch
import torch.nn as nn
from torch.autograd import Variable
import math

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_nodes = 30
        self.input_layer = nn.Linear(4, num_nodes)
        torch.nn.init.xavier_normal_(self.input_layer.weight)
        self.hidden_layer1 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer1.weight)
        self.hidden_layer2 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer2.weight)
        self.hidden_layer3 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer3.weight)
        self.output_layer = nn.Linear(num_nodes, 1)
        torch.nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, x, y, mu1, mu2):
        input = torch.cat([x, y, mu1, mu2],axis=1) # combines the column array
        layer1_out = torch.tanh(self.input_layer(input))
        layer2_out = torch.tanh(self.hidden_layer1(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer2(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer3(layer3_out))
        output = self.output_layer(layer4_out)
        return x * (1-y) * output


## PDE as loss function. Thus would use the network which we call as u_theta
def R(x, y, mu1, mu2, net):
    u = net(x, y, mu1, mu2)
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]

    u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]

    pde = - mu1 * (u_xx + u_yy) + y * (1-y) * u_x

    return pde

def Neumann_bottom(pt_x_bc_N_bottom, pt_y_bc_N_bottom, pt_mu1_N_bottom, pt_mu2_N_bottom, net):
    u = net(pt_x_bc_N_bottom, pt_y_bc_N_bottom, pt_mu1_N_bottom, pt_mu2_N_bottom)
    u_x = torch.autograd.grad(u.sum(), pt_x_bc_N_bottom, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), pt_y_bc_N_bottom, create_graph=True)[0]
    neumann = - pt_mu1_N_bottom * u_y - pt_mu2_N_bottom
    return neumann

def Neumann_right(pt_x_bc_N_right, pt_y_bc_N_right, pt_mu1_N_right, pt_mu2_N_right, net):
    u = net(pt_x_bc_N_right,pt_y_bc_N_right, pt_mu1_N_right, pt_mu2_N_right)
    u_x = torch.autograd.grad(u.sum(), pt_x_bc_N_right, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), pt_y_bc_N_right, create_graph=True)[0]
    neumann = u_x
    return neumann

def compute_l2_loss(w):
    return torch.square(w).sum()

from numpy.core.multiarray import ndarray
import matplotlib.pyplot as plt
from scipy.stats import qmc

np.random.seed(2)
random_seed = 1
torch.manual_seed(random_seed)

n_N_bottom = 2000
n_N_right = 2000
num_points = 4000

tol = 1.0e-07,
mtol = 1. - 1.0e-07,


# Neuman: right
x_bc_N_right = np.ones((n_N_right,1))
y_bc_N_right = np.ravel(np.linspace(tol, mtol, num = n_N_right)).reshape(-1,1)


# Neumann: bottom
x_bc_N_bottom = np.ravel(np.linspace(tol, mtol, num = n_N_bottom)).reshape(-1,1)
y_bc_N_bottom = np.zeros((n_N_bottom ,1))

### (2) Model
net = Net()
mse_cost_function = torch.nn.MSELoss() # Mean squared error
#optimizer = torch.optim.Adam(net.parameters())

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
#lambda1 = lambda epoch: 0.7 ** math.floor(epoch / 5000)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

### (3) Training / Fitting

print("######### TRAIN: PINN Toy #############")
loss_vector = []
lrs = []

iterations = 30000
for epoch in range(iterations):
    optimizer.zero_grad() # to make the gradients zero

    # Loss based on boundary conditions: Neumann right
    minibatch=train_wt.sample(n = n_N_right, replace=True)
    mu1_N_right = np.ravel(np.array(minibatch.mu1)).reshape(-1,1)
    mu2_N_right = np.ravel(np.array(minibatch.mu2)).reshape(-1,1)
    pt_mu1_N_right = Variable(torch.from_numpy(mu1_N_right).float(), requires_grad=False)
    pt_mu2_N_right = Variable(torch.from_numpy(mu2_N_right).float(), requires_grad=False)

    pt_x_bc_N_right = Variable(torch.from_numpy(x_bc_N_right).float(), requires_grad=True)
    pt_y_bc_N_right = Variable(torch.from_numpy(y_bc_N_right).float(), requires_grad=True)

    net_neumann_right = Neumann_right(pt_x_bc_N_right, pt_y_bc_N_right, pt_mu1_N_right, pt_mu2_N_right, net)
    zero_n_right = np.zeros((n_N_right , 1))
    pt_zero_n_right = Variable(torch.from_numpy(zero_n_right).float(), requires_grad=False)
    mse_bc_N_right = mse_cost_function(net_neumann_right, pt_zero_n_right)

    # Loss based on boundary conditions: Neumann bottom
    minibatch=train_wt.sample(n = n_N_bottom, replace=True)
    mu1_N_bottom = np.ravel(np.array(minibatch.mu1)).reshape(-1,1)
    mu2_N_bottom = np.ravel(np.array(minibatch.mu2)).reshape(-1,1)
    pt_mu1_N_bottom = Variable(torch.from_numpy(mu1_N_bottom).float(), requires_grad=False)
    pt_mu2_N_bottom = Variable(torch.from_numpy(mu2_N_bottom).float(), requires_grad=False)

    pt_x_bc_N_bottom = Variable(torch.from_numpy(x_bc_N_bottom).float(), requires_grad=True)
    pt_y_bc_N_bottom = Variable(torch.from_numpy(y_bc_N_bottom).float(), requires_grad=True)

    net_neumann_bottom = Neumann_bottom(pt_x_bc_N_bottom, pt_y_bc_N_bottom, pt_mu1_N_bottom, pt_mu2_N_bottom, net)
    zero_n_bottom = np.zeros((n_N_bottom ,1))
    pt_zero_n_bottom = Variable(torch.from_numpy(zero_n_bottom).float(), requires_grad=False)
    mse_bc_N_bottom = mse_cost_function(net_neumann_bottom, pt_zero_n_bottom)

    # Loss based on PDE
    minibatch=train_wt.sample(n = num_points, replace=True)
    mu1_collocation = np.ravel(np.array(minibatch.mu1)).reshape(-1,1)
    mu2_collocation = np.ravel(np.array(minibatch.mu2)).reshape(-1,1)
    pt_mu1_collocation = Variable(torch.from_numpy(mu1_collocation).float(), requires_grad=False)
    pt_mu2_collocation = Variable(torch.from_numpy(mu2_collocation).float(), requires_grad=False)

    x_collocation = np.random.uniform(low=tol, high=mtol, size=(num_points,1))
    y_collocation = np.random.uniform(low=tol, high=mtol, size=(num_points,1))
    all_zeros = np.zeros((num_points,1))

    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True)
    pt_y_collocation = Variable(torch.from_numpy(y_collocation).float(), requires_grad=True)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False)

    f_out = R(pt_x_collocation, pt_y_collocation, pt_mu1_collocation, pt_mu2_collocation, net) # output of R(x,t)
    mse_f = mse_cost_function(f_out, pt_all_zeros)

    # Combining the loss functions
    loss = mse_bc_N_bottom + mse_bc_N_right + mse_f

    #l2_lambda = 0.0001
    #l2_regularization = torch.tensor(0., requires_grad=True)
    #for name, param in net.named_parameters():
    #    if 'bias' not in name:
    #      with torch.no_grad():
    #        l2_regularization += torch.sum(param*param)
    #loss += l2_lambda * l2_regularization

    loss.backward()
    optimizer.step()
    loss_vector.append(loss.item())
    #lrs.append(optimizer.param_groups[0]["lr"])


    with torch.autograd.no_grad():
      if( epoch % 200 == 0):
        print(epoch,"Loss:",loss.item(), "lr:", optimizer.param_groups[0]["lr"] )

    #scheduler.step()

torch.save(net.state_dict(), 'model_30000_NN_2000_Col_4000_wr_toy.pt')
df = pd.DataFrame(dict(loss=loss_vector))
df.describe()

df.to_csv("data_nn_30000_NN_2000_Col_4000_wr_toy.csv",sep = ';', header=True)
#files.download('./data_nn_30000_NN_2000_Col_4000_wr_toy.csv')

In [15]:
df = pd.read_csv('/content/Project_MOR/data_nn_30000_NN_2000_Col_4000_wr.csv', sep=';')
loss_vector = df['loss']
fig = px.scatter()
fig.add_scatter(x=np.arange(0, len(loss_vector)), y=loss_vector, name= "loss")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_yaxes(type="log")

fig.show()

FileNotFoundError: ignored

In [ ]:
M = 100

np.random.seed(1)

minibatch=train_wt.sample(n = M)
minibatch.describe()

,Unnamed: 0,mu1,mu2,ratio,ratio_abs
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1607.380000,1.097100,-0.018574,-0.029630,0.346075
std,985.834659,0.693000,0.447016,0.439556,0.270404
min,6.000000,0.117189,-0.948148,-0.929032,0.007750
25%,733.000000,0.651552,-0.248013,-0.274543,0.111174
50%,1618.500000,0.992523,-0.011526,-0.018974,0.282007
75%,2395.000000,1.340938,0.193491,0.281697,0.557227
max,3590.000000,3.608616,0.987704,0.995144,0.995144


In [ ]:
from scipy.sparse.linalg import splu

### define the training set

np.random.seed(1)

training_set = np.concatenate((np.ravel(np.array(minibatch['mu1'])).reshape(-1,1), np.ravel(np.array(minibatch['mu2'])).reshape(-1,1)), axis=1)

def normX(v, X):
	return np.sqrt(np.transpose(v) @ X @ v)

def ProjectSystem(AQH, fQH, B):
    AQN = []
    fQN = []
    for AH in AQH:
        AQN.append(np.copy(np.transpose(B) @ AH @ B))
    for fH in fQH:
        fQN.append(np.copy(np.transpose(B) @ fH))
    return [AQN, fQN]

def Solve_full_order(AQH, fQH, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQH[0]
    f = thetaF_mu[0] * fQH[0]
    for i in range(1, len(AQH)):
        A += thetaA_mu[i] * AQH[i]
    for i in range(1, len(fQH)):
        f += thetaF_mu[i] * fQH[i]
    return gedim.LUSolver(A, f, lib)

def Solve_reduced_order(AQN, fQN, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQN[0]
    f = thetaF_mu[0] * fQN[0]
    for i in range(1, len(AQN)):
        A += thetaA_mu[i] * AQN[i]
    for i in range(1, len(fQN)):
        f += thetaF_mu[i] * fQN[i]
    return np.linalg.solve(A, f)

def OfflineResidual(AQH, fQH, B, invX):
    Cq1q2 = []
    dq1q2 = []
    Eq1q2 = []

    for q1 in range(0, len(AQH)):
        Z = invX.solve(AQH[q1] @ B)

        aqh_list = []
        for q2 in range(0, len(AQH)):
            aqh_list.append(np.copy(np.transpose(Z) @ AQH[q2] @ B))
        Eq1q2.append(aqh_list.copy())

        fqh_list = []
        for q2 in range(0, len(fQH)):
            fqh_list.append(np.copy(np.transpose(Z) @ fQH[q2]))
        dq1q2.append(fqh_list.copy())

    for q1 in range(0, len(fQH)):
        t = invX.solve(fQH[q1])

        fqh_list = []
        for q2 in range(0, len(fQH)):
            fqh_list.append(np.copy(np.transpose(t) @ fQH[q2]))
        Cq1q2.append(fqh_list.copy())

    return [Cq1q2, dq1q2, Eq1q2]

def InfSupConstant(mu):
    return thetaA(mu)[0]

def ErrorEstimate(Cq1q2, dq1q2, Eq1q2, thetaA_mu, thetaF_mu, solN, beta):
    fError = 0.0
    for q1 in range(0, len(Cq1q2)):
        for q2 in range(0, len(Cq1q2[q1])):
            fError += thetaF_mu[q1] * thetaF_mu[q2] * Cq1q2[q1][q2]

    uError = 0.0
    for q1 in range(0, len(Eq1q2)):
        for q2 in range(0, len(Eq1q2[q1])):
            uError += thetaA_mu[q1] * thetaA_mu[q2] * np.transpose(solN) @ Eq1q2[q1][q2] @ solN

    fuError = 0.0
    for q1 in range(0, len(dq1q2)):
        for q2 in range(0, len(dq1q2[q1])):
            fuError += thetaA_mu[q1] * thetaF_mu[q2] * np.transpose(solN) @ dq1q2[q1][q2]

    deltaN_squared = fError - 2.0 * fuError + uError
    if abs(deltaN_squared) <= 1.0e-12: # protect cancellation error
        deltaN_squared = 0.0
    if deltaN_squared < -1.0e-12:
      print(deltaN_squared)
      raise Exception('deltaN_squared is negative')

    return np.sqrt(deltaN_squared) / beta

def GramSchmidt(V, u, X):
    z = u
    if np.size(V) > 0:
        z = u - V @ (np.transpose(V) @ (X @ u))
    return z / normX(z, X)

chosen_wt_mu = []

##### Greedy #####
def Greedy(AQH, fQH, X, N_max, tol):
    N = 0
    basis_functions = []
    B = np.empty((0,0))
    deltaN = tol + 1.
    training_set_list = training_set.tolist()
    initial_muN = np.random.choice(len(training_set_list) - 1, 1)[0]
    muN = training_set_list.pop(initial_muN)
    invX = splu(X)

    print('Perfom greedy algorithm...')
    while len(training_set_list) > 0 and N < N_max and deltaN > tol:
        N = N + 1
        chosen_wt_mu.append(muN)
        print('\t', N,'/', N_max, '-', '{:.16e}'.format(np.mean(deltaN)), '/', '{:.16e}'.format(np.mean(tol)))
        snapshot = Solve_full_order(AQH, fQH, thetaA(muN), thetaF(muN))
        basis_function = GramSchmidt(B, snapshot, X)
        basis_functions.append(np.copy(basis_function))
        B = np.transpose(np.array(basis_functions))
        BX = np.transpose(B) @ X @ B

        [AQN, fQN] = ProjectSystem(AQH, fQH, B)
        [Cq1q2, dq1q2, Eq1q2] = OfflineResidual(AQH, fQH, B, invX)

        counter = 0
        mu_selected_index = -1
        max_deltaN = -1.
        for mu in training_set_list:
            solN_mu = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
            betaN_mu = InfSupConstant(mu)
            deltaN_mu = ErrorEstimate(Cq1q2, dq1q2, Eq1q2, thetaA(mu), thetaF(mu), solN_mu, betaN_mu)

            if deltaN_mu > max_deltaN:
                max_deltaN = deltaN_mu
                mu_selected_index = counter

            counter = counter + 1

        if mu_selected_index == -1:
            raise Exception('ERROR, parameter not found')

        muN = training_set_list.pop(mu_selected_index)
        deltaN = max_deltaN

    return [N, np.transpose(np.array(basis_functions))]

### Compute Greedy
tol = 1.0e-5
N_max = 40

[N_Greedy, B_Greedy] = Greedy(AQH, fQH, X, N_max, tol)
print("N_Greedy", N_Greedy)

[AQN_Greedy, fQN_Greedy] = ProjectSystem(AQH, fQH, B_Greedy)

Perfom greedy algorithm...
	 1 / 40 - 1.0000100000000001e+00 / 1.0000000000000001e-05
	 2 / 40 - 2.1016978051865198e-02 / 1.0000000000000001e-05
	 3 / 40 - 7.9326678042240730e-04 / 1.0000000000000001e-05
	 4 / 40 - 1.0528089993571229e-05 / 1.0000000000000001e-05
N_Greedy 4


In [ ]:
print(np.array(chosen_wt_mu)[:,1]/ np.array(chosen_wt_mu)[:,0])

[-0.56781055  0.99514443  0.49512994 -0.79840226]


In [ ]:
print(B_Greedy.shape)

(7757, 4)


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_nodes = 30
        self.input_layer = nn.Linear(4, num_nodes)
        torch.nn.init.xavier_normal_(self.input_layer.weight)
        self.hidden_layer1 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer1.weight)
        self.hidden_layer2 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer2.weight)
        self.hidden_layer3 = nn.Linear(num_nodes, num_nodes)
        torch.nn.init.xavier_normal_(self.hidden_layer3.weight)
        self.output_layer = nn.Linear(num_nodes, 1)
        torch.nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, x, y, mu1, mu2):
        input = torch.cat([x, y, mu1, mu2],axis=1) # combines the column array
        layer1_out = torch.tanh(self.input_layer(input))
        layer2_out = torch.tanh(self.hidden_layer1(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer2(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer3(layer3_out))
        output = self.output_layer(layer4_out)
        return x * (1-y) * output

net = Net()
net.load_state_dict(torch.load('/content/Project_MOR/model_30000_NN_2000_Col_4000_wr_toy.pt',))
net.eval()

Net(
  (input_layer): Linear(in_features=4, out_features=30, bias=True)
  (hidden_layer1): Linear(in_features=30, out_features=30, bias=True)
  (hidden_layer2): Linear(in_features=30, out_features=30, bias=True)
  (hidden_layer3): Linear(in_features=30, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=1, bias=True)
)

In [ ]:
test_wt = pd.DataFrame(dict(mu1 = test.mu1, mu2 = test.mu2, ratio = test.ratio, ratio_abs=test.ratio_abs))
test_wt.describe()

,mu1,mu2,ratio,ratio_abs
count,752.000000,752.000000,752.000000,752.000000
mean,0.872130,-0.070717,-0.111107,0.981159
std,0.681209,0.542968,1.615976,1.288326
min,0.117189,-0.973965,-8.090738,0.000007
25%,0.349802,-0.480510,-0.653605,0.189687
50%,0.776945,-0.126505,-0.088095,0.552062
75%,1.083176,0.251904,0.370512,1.196361
max,3.608616,0.987704,8.075570,8.090738


In [ ]:
from numpy import linalg
import math
import time

num_mu1 = len(test_wt.mu1)

errors_PINN = np.zeros(num_mu1)
errors_inf_PINN = np.zeros(num_mu1)
speed_up_PINN = np.zeros(num_mu1)

errors_ROM = np.zeros(num_mu1)
errors_inf_ROM = np.zeros(num_mu1)
speed_up_ROM = np.zeros(num_mu1)

max_abs_NN_sol = np.zeros(num_mu1)
max_NN_sol = np.zeros(num_mu1)
min_NN_sol = np.zeros(num_mu1)

max_abs_FOM_sol = np.zeros(num_mu1)
max_FOM_sol = np.zeros(num_mu1)
min_FOM_sol = np.zeros(num_mu1)

max_abs_ROM_sol = np.zeros(num_mu1)
max_ROM_sol = np.zeros(num_mu1)
min_ROM_sol = np.zeros(num_mu1)

df_mesh = pd.read_csv('/content/Project_MOR/mesh0001.csv', sep=';')

count = 0
for (mu1,mu2) in zip(test_wt.mu1, test_wt.mu2):

  mu = [mu1, mu2]
  x = np.array(df_mesh[df_mesh.columns[1]])
  y = np.array(df_mesh[df_mesh.columns[2]])

  # PINN solution
  x_reshape = np.ravel(x).reshape(-1,1)
  y_reshape = np.ravel(y).reshape(-1,1)

  start_PINN = time.time()
  pt_mu1_test = Variable(torch.from_numpy(mu1 * np.ones((x.shape[0], 1))).float(), requires_grad=True)
  pt_mu2_test = Variable(torch.from_numpy(mu2 * np.ones((x.shape[0], 1))).float(), requires_grad=True)
  pt_x = Variable(torch.from_numpy(x_reshape).float(), requires_grad=True)
  pt_y = Variable(torch.from_numpy(y_reshape).float(), requires_grad=True)
  pt_u = net(pt_x,pt_y, pt_mu1_test, pt_mu2_test)
  time_PINN = time.time() - start_PINN

  nn_solution = np.squeeze(pt_u.detach().numpy())
  max_abs_NN_sol[count] = np.absolute(nn_solution).max()
  max_NN_sol[count] = nn_solution.max()
  min_NN_sol[count] = nn_solution.min()

  ##### full #####
  start_fom = time.time()
  full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))

  fem_solution = np.zeros(x.shape)
  count_dof = 0
  count_strong = 0
  for i in range(len(fem_solution)):
      if abs(x[i]) < geometricTolerance or abs(y[i] - 1.) < geometricTolerance:
          fem_solution[i] = 0.
          count_strong += 1
      else:
          fem_solution[i] = full_solution[count_dof]
          count_dof += 1

  time_fom = time.time() - start_fom

  max_abs_FOM_sol[count] = np.absolute(fem_solution).max()
  max_FOM_sol[count] = fem_solution.max()
  min_FOM_sol[count] = fem_solution.min()


  #### reduced #####
  start_rom = time.time()
  reduced_solution = Solve_reduced_order(AQN_Greedy, fQN_Greedy, thetaA(mu), thetaF(mu))

  proj_reduced_solution = B_Greedy @ reduced_solution

  rom_solution = np.zeros(x.shape)
  count_dof = 0
  count_strong = 0
  for i in range(len(rom_solution)):
      if abs(x[i]) < geometricTolerance or abs(y[i] - 1.) < geometricTolerance:
          rom_solution[i] = 0.
          count_strong += 1
      else:
          rom_solution[i] = proj_reduced_solution[count_dof]
          count_dof += 1

  time_rom = time.time() - start_rom

  max_abs_ROM_sol[count] = np.absolute(rom_solution).max()
  max_ROM_sol[count] = rom_solution.max()
  min_ROM_sol[count] = rom_solution.min()


  # compute errors
  speed_up_PINN[count] = time_fom / time_PINN
  errors_PINN[count] = linalg.norm(nn_solution - fem_solution)**2 / len((fem_solution))
  errors_inf_PINN[count] = linalg.norm((nn_solution - fem_solution), ord = np.inf)

  speed_up_ROM[count] = time_fom / time_rom
  errors_ROM[count] = linalg.norm(proj_reduced_solution - full_solution)**2 / len((full_solution))
  errors_inf_ROM[count] = linalg.norm((proj_reduced_solution - full_solution), ord = np.inf)

  count +=1

In [ ]:
d_wt = {'MSE_PINN': errors_PINN, 'Err_Inf_PINN': errors_inf_PINN, 'Speed_Up_PINN': speed_up_PINN,'MSE_ROM': errors_ROM, 'Err_Inf_ROM': errors_inf_ROM, 'Speed_Up_ROM': speed_up_ROM}
df_wt = pd.DataFrame(data=d_wt)

df_wt.describe().apply(lambda s: s.apply('{0:.2e}'.format))

,MSE_PINN,Err_Inf_PINN,Speed_Up_PINN,MSE_ROM,Err_Inf_ROM,Speed_Up_ROM
count,7.52e+02,7.52e+02,7.52e+02,7.52e+02,7.52e+02,7.52e+02
mean,3.34e-02,1.46e-01,1.23e+01,4.14e-14,8.69e-08,5.67e+00
std,1.72e-01,4.14e-01,2.43e+00,2.85e-13,4.49e-07,9.93e-01
min,4.43e-08,9.32e-04,3.16e+00,1.66e-37,8.10e-19,2.24e+00
25%,3.45e-06,6.70e-03,1.17e+01,3.11e-19,1.18e-09,5.56e+00
50%,2.07e-05,1.05e-02,1.29e+01,6.50e-18,5.37e-09,5.90e+00
75%,5.92e-04,6.46e-02,1.36e+01,5.76e-17,1.60e-08,6.11e+00
max,2.06e+00,3.50e+00,3.39e+01,3.67e-12,4.30e-06,9.49e+00


In [ ]:
test_wt['max_FOM'] = max_FOM_sol
test_wt['min_FOM'] = min_FOM_sol
test_wt['max_FOM_abs'] = max_abs_FOM_sol

fig = px.scatter()
fig.add_scatter(x=test_wt.ratio, y=test_wt.min_FOM, mode='markers', name= "min")
fig.add_scatter(x=test_wt.ratio, y=test_wt.max_FOM, mode='markers', name= "max")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    xaxis_title="mu2/mu1",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [ ]:
test_wt['max_PINN'] = max_NN_sol
test_wt['min_PINN'] = min_NN_sol
test_wt['max_PINN_abs'] = max_abs_NN_sol

fig = px.scatter()
fig.add_scatter(x=test_wt.ratio, y=test_wt.min_PINN, mode='markers', name= "min")
fig.add_scatter(x=test_wt.ratio, y=test_wt.max_PINN, mode='markers', name= "max")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    xaxis_title="mu2/mu1",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [ ]:
test_wt['max_ROM'] = max_ROM_sol
test_wt['min_ROM'] = min_ROM_sol
test_wt['max_ROM_abs'] = max_abs_ROM_sol

fig = px.scatter()
fig.add_scatter(x=test_wt.ratio, y=test_wt.min_ROM, mode='markers', name= "min")
fig.add_scatter(x=test_wt.ratio, y=test_wt.max_ROM, mode='markers', name= "max")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    xaxis_title="mu2/mu1",
    font=dict(
        family="Courier New, monospace",
        size=25
    )
)
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()